In [1]:
# coding: utf-8
# ラジコン自走コード
# pip install futures

import time
from concurrent import futures
import os
from multiprocessing import Manager
import thread
import threading
import logging
from __future__ import division
import Adafruit_PCA9685
import smbus
import numpy as np
import LidarLiteV3
import tensorflow as tf


# ログ設定
logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] time:%(created).8f pid:%(process)d pn:%(processName)-10s tid:%(thread)d tn:%(threadName)-10s fn:%(funcName)-10s %(message)s',
)

LIDAR_INTERVAL = 0.05

'''
ステアリング制御クラス
'''
class SteeringControl():
    def __init__(self, calibration=0):
        self.CALIBRATION = calibration

        self.pwm = Adafruit_PCA9685.PCA9685()
        self.pwm.set_pwm_freq(60)

        self.HANDLE_NUTRAL = 375 + self.CALIBRATION
        #self.HANDLE_MAX_RIGHT = 150
        #self.HANDLE_MAX_LEFT = 600

        self.MAX_HANDLE_ANGLE = 45
        self.HANDLE_RIGHT = self.HANDLE_NUTRAL + self.MAX_HANDLE_ANGLE
        self.HANDLE_LEFT = self.HANDLE_NUTRAL - self.MAX_HANDLE_ANGLE

        self.channel = 0
        return

    def right(self):
        self.pwm.set_pwm(self.channel, 0, self.HANDLE_RIGHT)
    def forward(self):
        self.pwm.set_pwm(self.channel, 0, self.HANDLE_NUTRAL)
    def left(self):
        self.pwm.set_pwm(self.channel, 0, self.HANDLE_LEFT)
    def stop(self):
        self.pwm.set_pwm(self.channel, 0, self.HANDLE_NUTRAL)


'''
モーター制御クラス
'''
class MotorControl():
    def __init__(self):
        ## DRV8830 Default I2C slave address
        self.MOTOR1_ADDRESS  = 0x64
        ''' DRV8830 Register Addresses '''
        ## sample rate driver
        self.CONTROL = 0x00
        ## Value motor.
        self.FORWARD = 0x02
        self.BACK = 0x01
        self.STOP = 0x00

        self.MAX_SPEED = 101

        ## smbus
        self.bus = smbus.SMBus(1)

    def map(self, x, in_min, in_max, out_min, out_max):
        return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

    def drive(self, speed=101):
        if speed < 0:
            return
        if speed > self.MAX_SPEED:
            return
        s = self.map(speed, 1, 100, 1, 58)
        sval = self.FORWARD | ((s+5)<<2) #スピードを設定して送信するデータを1Byte作成
        self.bus.write_i2c_block_data(self.MOTOR1_ADDRESS,self.CONTROL,[sval]) #生成したデータを送信

    def back(self, speed=101):
        if speed < 0:
            return
        if speed > self.MAX_SPEED:
            return
        s = self.map(speed, 1, 100, 1, 58)
        sval = self.BACK | ((s+5)<<2) #スピードを設定して送信するデータを1Byte作成
        self.bus.write_i2c_block_data(self.MOTOR1_ADDRESS,self.CONTROL,[sval]) #生成したデータを送信

    def stop(self):
        speed = 0
        s = self.map(speed, 1, 100, 1, 58)
        sval = self.FORWARD | ((s+5)<<2) #スピードを設定して送信するデータを1Byte作成
        self.bus.write_i2c_block_data(self.MOTOR1_ADDRESS,self.CONTROL,[sval]) #生成したデータを送信


'''
ここはプロセスで実行される
SHARED_VARIABLE['CONTROL_READY']=True であるうちは実行を続ける
'''
def do_control():
    logging.debug("enter")
    SHARED_VARIABLE['CONTROL_READY']=True

    STOP=0
    LEFT=1
    FORWARD=2
    RIGHT=3

    calibration = +0
    steering_control = SteeringControl(calibration=calibration)
    motor_control = MotorControl()
    # タイヤをまっすぐにする
    steering_control.forward()

    ####################
    # ループ実行
    ####################
    while SHARED_VARIABLE['CONTROL_READY']:
        print("control: ", SHARED_VARIABLE['PREDICTION_VALUE'])
        if SHARED_VARIABLE['PREDICTION_VALUE'] == STOP:
            motor_control.stop()
            steering_control.forward()
        if SHARED_VARIABLE['PREDICTION_VALUE'] == LEFT:
            steering_control.left()
            motor_control.drive()
        if SHARED_VARIABLE['PREDICTION_VALUE'] == FORWARD:
            steering_control.forward()
            motor_control.drive()
        if SHARED_VARIABLE['PREDICTION_VALUE'] == RIGHT:
            steering_control.right()
            motor_control.drive()

        time.sleep(LIDAR_INTERVAL)

    motor_control.stop()
    steering_control.forward()
    return



def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph


'''
ここはプロセスで実行される
SHARED_VARIABLE['PREDICTION_READY']=True であるうちは実行を続ける
'''
def do_prediction():
    logging.debug("enter")
    SHARED_VARIABLE['PREDICTION_READY']=True
    tf.reset_default_graph()

    ########################################
    # AI準備
    ########################################
    try:

        # Lidar準備
        lidar1 = LidarLiteV3.Connect(0x52)
        lidar2 = LidarLiteV3.Connect(0x54)
        lidar3 = LidarLiteV3.Connect(0x56)
        
        # AIモデルファイル名とディレクトリ
        FROZEN_MODEL_NAME="car_lidar_queue_20000.pb"
        MODEL_DIR = "../model_car_lidar_queue"

        # AIモデル読み込み
        graph = load_graph(MODEL_DIR+"/"+FROZEN_MODEL_NAME)
        graph_def = graph.as_graph_def()
        # AI入出力ノード取得
        input_x = graph.get_tensor_by_name('prefix/queue/dequeue_op:0')
        output_y= graph.get_tensor_by_name('prefix/neural_network_model/output_y:0')

        n_classes=4 # 出力数(STOP,LEFT,FORWARD,RIGHTの4種類)
        start_time, start_clock = time.time(), time.clock()

        '''
        AI予測実行
        sensor:[[100,200,100]] # [[LEFT45,FRONT,RIGHT45]], int型[1,3]配列
        max_index:N # 0:STOP,1:LEFT,2:FORWARD,3:RIGHT int型
        '''
        with tf.Session(graph=graph) as sess:
            ####################
            # ループ実行
            ####################
            while SHARED_VARIABLE['PREDICTION_READY']:
                distance1 = lidar1.getDistance()
                distance2 = lidar2.getDistance()
                distance3 = lidar3.getDistance()

                _output_y = sess.run(output_y,feed_dict={input_x:[[distance1,distance2,distance3]]})
                max_index = np.argmax(_output_y) # max_value's index no
                SHARED_VARIABLE['PREDICTION_VALUE'] = max_index

                time.sleep(LIDAR_INTERVAL)

    except Exception as e:
        print(str(e))
        if not SHARED_VARIABLE['PREDICTION_READY']:
            print('error! PREDICTION_READY is False')
    finally:
        return





'''
ここはプロセスで実行される
'''
def do_stop():
    tag='STOP'

    ####################
    # ループ実行
    ####################
    start_time = time.time()
    RUNNING_SEC = 30
    time.sleep(RUNNING_SEC)
    SHARED_VARIABLE['CONTROL_READY']=False
    SHARED_VARIABLE['PREDICTION_READY']=False
    SHARED_VARIABLE['IF_READY']=False

    return

'''
process pattern
'''
SHARED_VARIABLE=Manager().dict()
SHARED_VARIABLE['CONTROL_READY']=False
SHARED_VARIABLE['PREDICTION_READY']=False
SHARED_VARIABLE['IF_READY']=False
SHARED_VARIABLE['CONTROL_VALUE']=0
SHARED_VARIABLE['PREDICTION_VALUE']=0

'''
プロセスによる実行関数の振り分け定義
'''
PROCESS_LIST=['do_control','do_prediction','do_stop']
def do_process(target):

    if target == 'do_control':
        do_control()
        return "end do_control"
    if target == 'do_prediction':
        do_prediction()
        return "end do_prediction"
    if target == 'do_stop':
        do_stop()
        return "end do_stop"

'''
メイン処理を行う部分
・メインスレッド（ここ）
・スレッド1(concurrent.futures)
・スレッド2(concurrent.futures)
・制御スレッド(concurrent.futures)
'''
def do_main():
    try:
        with futures.ProcessPoolExecutor(max_workers=len(PROCESS_LIST)) as executer:
            mappings = {executer.submit(do_process, pname): pname for pname in PROCESS_LIST}
            for i in futures.as_completed(mappings):
                target = mappings[i]
                result = i.result()
                print(result)

    except Exception as e:
        print('error! executer failed.')
        print(str(e))
    finally:
        print("executer end")

    return

do_main()


[DEBUG] time:1496114593.96136999 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:do_control enter
[DEBUG] time:1496114593.96842408 pid:28092 pn:Process-3  tid:1996107776 tn:MainThread fn:do_prediction enter
[DEBUG] time:1496114593.99765801 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0xFA
[DEBUG] time:1496114594.00250196 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0xFB
[DEBUG] time:1496114594.02501106 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0xFC
[DEBUG] time:1496114594.03811193 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0xFD
[DEBUG] time:1496114594.04474998 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x04 to register 0x01
[DEBUG] time:1496114594.05090499 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x01 to register 0x00
[DEBUG] time:1496

('control: ', 0)


[DEBUG] time:1496114594.14470005 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x06
[DEBUG] time:1496114594.14906001 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x07
[DEBUG] time:1496114594.15367508 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x77 to register 0x08
[DEBUG] time:1496114594.15852499 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x01 to register 0x09


('control: ', 0)


[DEBUG] time:1496114594.21954393 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x06
[DEBUG] time:1496114594.22478890 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x07
[DEBUG] time:1496114594.22981310 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x77 to register 0x08
[DEBUG] time:1496114594.23507500 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x01 to register 0x09


...

('control: ', 2)


[DEBUG] time:1496114623.87139297 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x06
[DEBUG] time:1496114623.87611103 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x07
[DEBUG] time:1496114623.88094211 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x77 to register 0x08
[DEBUG] time:1496114623.88608694 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x01 to register 0x09


('control: ', 1)


[DEBUG] time:1496114623.94711304 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x06
[DEBUG] time:1496114623.95217991 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x07
[DEBUG] time:1496114623.95747399 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x4A to register 0x08
[DEBUG] time:1496114623.96301603 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x01 to register 0x09
[DEBUG] time:1496114624.02020192 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x06
[DEBUG] time:1496114624.02521706 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x00 to register 0x07
[DEBUG] time:1496114624.03035593 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrote 0x77 to register 0x08
[DEBUG] time:1496114624.03515792 pid:28091 pn:Process-2  tid:1996107776 tn:MainThread fn:write8     Wrot

end do_stop
end do_control
end do_prediction
executer end
